## 1. Overview

In a perfect world, all data is presented in a clear, tabular format - easy for consumption and ready for analytics! In reality, data often lives in disparate sources, which requires us to collect and gather it first.

In this tutorial, I'll explore a scenario with Excel files: a company has customer information stored in a number of Excel forms. These forms are saved within a specific directory. We will be tasked with creating a final dataframe for all customer information using the values from these Excel files.

### 1.1. Import Packages

In [134]:
import os
import pandas as pd
import xlrd

#Set Directory with files
os.chdir('./Files')

### 1.2. Use xlrd to open and read Excel workbooks
**xlrd** is a package we can use to open and read Excel workbooks. The **open_workbook** function requires the file path of the workbook, let's use Form - Hopper, Grace to start. Opening the workbook will require an **xlrd.book.Book object**

In [122]:
wb = xlrd.open_workbook('Form - Hopper, Grace.xlsx')
print(wb)

### 1.3. Get worksheeets within Excel workbook
Excel workbooks can contain multiple worksheets. Before we can grab any data, we need to specify which worksheet we want to use within the workbook.

In xlrd, there are two ways we can call on a specific workbook:
- **By Index** we can use the index of the work sheet within the workbook with **sheet_by_index** function (0 indexed)
- **By Name** if we know the name of the work sheet, we can use the **sheey_by_name** function

In [123]:
#To get sheet by index (starts with 0)
ws = wb.sheet_by_index(0)
#To get sheet by name 
ws = wb.sheet_by_name('Sheet1')

### 1.4. Reading Data in an Excel worksheet

To read data in a worksheet, we can use either the **row_values** function to return values for a specific row or we can use **column_values** function to return valuese for a specific column. Each requires a numeric index argument.

In [127]:
#get list of values in row
ws.row_values(2)

['First Name', 'Grace']

In [124]:
#get list of values in column
ws.col_values(0)

['Information Fill Out Form',
 '',
 'First Name',
 'Last Name',
 'Sex',
 'DOB',
 'City',
 'State']

## 2. Use Case
Company ABC has a directory with various Excel files containing customer information (e.g. First Name, Last Name, City, State). The information is stored such that Column A contains all questions (attributes, e.g. First Name) and Column B contains all output values (e.g. Grace). 

Your manager wants to cataolgue and analyze this information for all customers. 

This means we need to create a scraping job to iterate through all Excel files in the directory, open the workbook, and scrape the pre-defined attributes. The end result should yield a dataframe where each row represents a customer, and each column represents an attribute or dimension describing that customer.

### 2.1 Grabbing Data from Columnar Key Values in a Worksheet

Let's start small. Before we get into iterating through files in a directory, let's try getting the value for one attribute in a file. 

We will use the example workbook for Grace Hopper in section 1.3. Let's write a code to return the First Name output.

In [32]:
#get the index of the attribute specified, e.g. First Name
attributes = ws.col_values(0)
attribute_index = attributes.index('First Name')

#get the neighboring value output of the attribute, e.g. Grace Hopper
values = ws.col_values(1)
values[attribute_index]

'Grace'

### 2.2. Let's Create a Function!

Doing this over and over for every attribute will get tedious. To reduce code, we can condense this by creating our own function.

For our purposes, we will assume that the value for each attribute is stored in the neighboring column.

In our function, let's also account for the possible scenario that an Excel worksheet might not have a listed attribute. In this instances, we will default the value output to **None**.

In [1]:
def get_value(worksheet, attribute_column, attribute_name):
    attributes = worksheet.col_values(attribute_column)
    if attribute_name in attributes:
        attribute_index = attributes.index(attribute_name)
        #assume value is in the adjacent column where attribute is stored
        values = worksheet.col_values(attribute_column+1)
        value = values[attribute_index]
        return value
    else:
        return None
        
get_value(ws,0,'First Name')

NameError: name 'ws' is not defined

### 2.3. Looping Through Files

Let's put it all together with a for loop. Up to this point we've reviewed how to pull attribute information for one file. To create our database of customer information, we will need to iterate or loop through all files in our directory. We can use the **os.walk** function to loop through each file.

In [130]:
for root, dirs, files, in os.walk('.'):
    for file in files:
        print(file)

Form - Smith, John.xlsx
Form - Hopper, Grace.xlsx
Form - Wayne, Bruce.xlsx
Form - Doe, Jane.xlsx


### 2.4. Putting it All Together - Loop & Scrape

For each file, we will need to scrape the attribute along with the corresponding value, e.g. First Name: Grace, Last Name: Hopper. An easy way to store this data is a dictionary, where the values are stored as a list.

We can initialize an empty dictionary at the beginning of the loop, all keys are equal to attribute names. For indexing purposes, it might be useful to include the **File Name**. We will append this to our empty dictionary.

Once we loop through the files, we will loop once more through the list of attributes to append each value (use our **get_value** function from 2.2) to the appropriate key.

The end will result in a dictionary of our attributes and values.

In [135]:
for root, dirs, files in os.walk('.'):
    attributes = ['First Name', 'Last Name', 'Sex','City','State']
    #initialized dictionary, create empty list for attributes with dict comprehension
    data = {attribute: [] for attribute in attributes}
    #append a key:value for File, will use this as unique identifier/index
    data.update({"File": []})
    for file in files:
        wb = xlrd.open_workbook(file)
        ws = wb.sheet_by_index(0)
        data['File'].append(file)
        for attribute in attributes:
            data[attribute].append(get_value(ws,0,attribute))

data

{'First Name': ['John', 'Grace', 'Bruce', 'Jane'],
 'Last Name': ['Smith', 'Hopper', 'Wayne', 'Doe'],
 'Sex': ['Male', 'Female', 'Male', 'Female'],
 'City': ['San Francisco', 'New York', 'Gotham City', 'Orlando'],
 'State': ['CA', 'NY', 'NJ', 'FL'],
 'File': ['Form - Smith, John.xlsx',
  'Form - Hopper, Grace.xlsx',
  'Form - Wayne, Bruce.xlsx',
  'Form - Doe, Jane.xlsx']}

### 2.5. Converting a Dictionary output to a Dataframe

Last but not least, we'll take the resulting dictionary and convert it to a dataframe. To do this we will use **pandas**.

In [136]:
df = pd.DataFrame.from_dict(data)
df.to_excel("Scraped_Data.xlsx",sheet_name="Sheet1")

,First Name,Last Name,Sex,City,State,File
0,John,Smith,Male,San Francisco,CA,"Form - Smith, John.xlsx"
1,Grace,Hopper,Female,New York,NY,"Form - Hopper, Grace.xlsx"
2,Bruce,Wayne,Male,Gotham City,NJ,"Form - Wayne, Bruce.xlsx"
3,Jane,Doe,Female,Orlando,FL,"Form - Doe, Jane.xlsx"
